In [ ]:
!pip install gym

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import gym
from ReplayBuffer import ReplayBuffer

import tensorflow as tf

In [ ]:
np.random.seed(seed=42)
tf.random.set_seed(42)

In [ ]:
env = gym.make("Acrobot-v1")



NUM_STATES = env.observation_space.shape[0]
NUM_ACTIONS = env.action_space.n

print(f'States: {NUM_STATES} all ot them continuius')
print(f'theta1 is the angle of the first joint')
print("theta2 is relative to the angle of the first link")
print('Actions: {}'.format(NUM_ACTIONS))

In [ ]:
lr = 0.1 # learning rate
gamma = 0.99 # параметр дисконтирования

NUM_EPISODES = 1000 # число эпизодов для обучения
MAX_STEPS = 500 # максимальное число шагов в эпизоде

REWARD_AVERAGE_WINDOW = 20 # окно для усреднения наград по эпизодам

In [ ]:
class Agent:
    def __init__(self, lr, gamma, NUM_ACTIONS,NUM_STATES, batch_size, epsilon=0.95, ):
        self.gamma = gamma
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.epsilon_decay = 0.001
        self.epsilon_final = 0.05
        self.update_rate = 100
        self.step_counter = 0
        self.buffer = ReplayBuffer(100000)
        self.q_net = Q_Model(lr, NUM_ACTIONS, NUM_STATES,)
        self.q_target_net = Q_Model(lr, NUM_ACTIONS, NUM_STATES, )
        
    
    def train_step(self):
        
        #update target network after n steps
        if self.step_counter % self.update_rate == 0:
            self.q_target_net.set_weights(self.q_net.get_weights())
        #sample env data from replay buffer
        state_batch, action_batch, reward_batch, new_state_batch, done_batch = \
            self.buffer.sample(self.batch_size)

        self.buffer.sample(self.batch_size)

        #predict q value based on main policy
        q_predicted = self.q_net(state_batch)
        q_target = np.copy(q_predicted)

        #predict next q value based on target policy to lower correlation between states
        q_next = self.q_target_net(new_state_batch)
        q_max_next = tf.math.reduce_max(q_next, axis=1, keepdims=True).numpy()
        #Bellman  Equation
        q_target[:, action_batch]=reward_batch+self.gamma*q_max_next*(np.logical_not(done_batch)).astype(int)
        self.q_net.train_on_batch(state_batch, q_target)
       # self.epsilon = self.epsilon - self.epsilon_decay if self.epsilon > self.epsilon_final else self.epsilon_final
        self.step_counter += 1
    def save_network(self,txt,i,num_episodes,score,avg_score):
        self.q_net.save((fr"saved_networks/dqn_model{i}"))
        self.q_net.save_weights((fr"saved_networks/dqn_model{i}/net_weights{i}.h5"))
        txt.write(fr"Save {i} - Episode {i}/{num_episodes}, Score: {score} ({self.epsilon}), AVG Score: {avg_score}\n")
        print("Network saved")
    def train_model(self, env, num_episodes, ):
        scores, episodes, avg_scores  = [], [], []
        txt = open("saved_networks.txt", "w")

        for i in range(num_episodes):
            done = False
            score = 0.0
            state = env.reset()[0]
            while not done and (score>-500):
                if np.random.rand() < self.epsilon:
                # explore
                    action = env.action_space.sample()
                else:
                # exploit
                   # print(state)
                    actions=self.q_net(tf.expand_dims(state,0))
                    action = tf.math.argmax(actions, axis=1).numpy()[0]
                new_state, reward, done, _,_ = env.step(action)
                #if len(new_state)>1:
                #    new_state=new_state[0]
                score += reward
                reward+=float(done)*100
                self.buffer.add(state, action, reward, new_state, done)
                state = new_state
                if len(self.buffer)>=3*self.batch_size:
                    self.train_step()
            if self.epsilon>self.epsilon_final:
                self.epsilon-=self.epsilon_decay

            scores.append(score)
            episodes.append(i)
            avg_score = np.mean(scores[-20:])
            avg_scores.append(avg_score)
            print(f"Episode {i}/{num_episodes}, Score: {score} ({self.epsilon}), AVG Score: {avg_score}")
            self.save_network(txt,i,num_episodes,score,avg_score)
        txt.close()




In [ ]:
dqn_agent =Agent(lr=0.1, gamma=0.99, NUM_ACTIONS=NUM_ACTIONS,NUM_STATES=NUM_STATES, epsilon=0.95, batch_size=32)
import warnings
warnings.filterwarnings('ignore')
dqn_agent.train_model(env, num_episodes=1000)